<a href="https://colab.research.google.com/github/jansoe/dl_workshop/blob/main/Squad_wt_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pretrained SQuAD Transformer


In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 1.5MB 5.7MB/s 
     |████████████████████████████████| 2.9MB 19.7MB/s 
     |████████████████████████████████| 890kB 45.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=980d96d267b45839ba09065c5381a38233ba8214ebbfc57f0b34e4ba978d1712
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 163kB 4.3MB/s 
     |████████████████████████████████| 245kB 31.5MB/s 
     |████████████████████████████████| 17.7MB 206kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [2]:
from pprint import pprint
import os
import torch
import torch.utils.data as datatools
import transformers

Sollten Sie Interesse daran haben, welche Hardware Ihnen von Google zur Verfügung gestellt wird, können Sie die folgende Code-Zelle ausführen.

In [11]:
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

##  Model

In [8]:
model_name_or_path = 'distilbert-base-uncased-distilled-squad' 
# model_name_or_path = 'bert-large-uncased-whole-word-masking-finetuned-squad'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name_or_path)

##  Inference

In [9]:
def evaluate(model, context, question):
    inputs = tokenizer.encode_plus(
        question, 
        context, 
        return_attention_mask = False, 
        return_tensors = 'pt',
        return_token_type_ids = not('distilbert' in model_name_or_path) 
    )

    # prediction for every token if it is start or end token
    out = model(**inputs)

    # we convert tokens to ids
    all_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].numpy().squeeze())

    # and combine all tokens from start to end
    answer = ' '.join(all_tokens[torch.argmax(out.start_logits) : torch.argmax(out.end_logits)+1])
    answer = answer.replace(' ##', '')
    return answer

In [10]:
context = """\
The Rhine (Romansh: Rein, German: Rhein, French: le Rhin, Dutch: Rijn) \
is a European river that begins in the Swiss canton of Graubünden in the southeastern Swiss Alps, \
forms part of the Swiss-Austrian, Swiss-Liechtenstein border, Swiss-German and then the Franco-German border, \
then flows through the Rhineland and eventually empties into the North Sea in the Netherlands. \
The biggest city on the river Rhine is Cologne, Germany with a population of more than 1,050,000 people. \
It is the second-longest river in Central and Western Europe (after the Danube), at about 1,230 km (760 mi),\
[note 2][note 1] with an average discharge of about 2,900 m3/s (100,000 cu ft/s).\
"""

question =  "Where does the Rhine begin?"
# question = "What is the largest city the Rhine runs through? "
# question = "What river is larger than the Rhine?"

evaluate(model=model, context=context, question=question)

'swiss canton of graubunden'

## Task
Play around with the model and different questions/contexts